# Evolutionary optimization to fMRI and EEG data

This notebook runs four different optimizations. It simulates the ALN model and runs a multi-objective evolutionary optimization algorithm called the NGSA-II algorithm in `neurolib`. 
The different runs are:

* 1) use fMRI and EEG data simulateneously, apply no filtering (down-to-up solutions)
* 2) use fMRI and EEG data simulateneously, apply filtering on the median firing rate (up-to-down solutions)
* 3) use fMRI only without adaptation (b=0)
* 4) use fMRI only with adaptation (b>=0)

No plots are drawn here. The results of these runs (saved as *.dill files) will be processed and plotted in another notebook.

In [4]:
# change to the root directory of the project
import os
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')
    
# This will reload all imports as soon as the code changes
%load_ext autoreload
%autoreload 2   

In [5]:
%aimport

Modules to reload:


Modules to skip:



In [6]:
try:
    import matplotlib.pyplot as plt
except ImportError:
    import sys
    !{sys.executable} -m pip install matplotlib seaborn
    import matplotlib.pyplot as plt
    
import numpy as np
import logging 

from neurolib.models.aln import ALNModel
from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.evolution import Evolution
import neurolib.utils.functions as func
import neurolib.utils.devutils as du

import neurolib.optimize.evolution.deapUtils as deapUtils

from neurolib.utils.loadData import Dataset
ds = Dataset("gw_big")

# a nice color map
plt.rcParams['image.cmap'] = 'plasma'

# Initialize model

In [24]:
model = ALNModel(Cmat = ds.Cmat, Dmat = ds.Dmat) # simulates the whole-brain model in 10s chunks by default if bold == True
# Resting state fits
model.params['dt'] = 0.1
model.params['duration'] = 10 * 60 * 1000 #ms
# testing: aln.params['duration'] = 0.2 * 60 * 1000 #ms
# real: aln.params['duration'] = 1.0 * 60 * 1000 #ms
model.params['save_dt'] = 10.0 # 10 ms sampling steps for saving data, should be multiple of dt

MainProcess root INFO     aln: Model initialized.


In [25]:
# structural values
model.params["Ke_gl"] = 300.0
model.params["signalV"] = 20.0

## Load EEG data

In [134]:
import dill
f_eeg, mean_eeg_power = dill.load(open("./data/mean_eeg_power_N3.dill", "rb"))

In [137]:
MEDIAN_KILLER = True

def evaluateSimulation(traj):
    rid = traj.id
    model = evolution.getModelFromTraj(traj)
    model.randomICs()
    defaultDuration = model.params['duration']
    invalid_result = (-np.inf, np.inf, -np.inf, )
    # -------- stage wise simulation --------
    
    # Stage 1 : simulate for a few seconds to see if there is any activity
    # ---------------------------------------
    model.params['duration'] = 11*1000.
    model.run()
    max_amp_output = np.max(
          np.max(model.output[:, model.t > 1000], axis=1) 
        - np.min(model.output[:, model.t > 1000], axis=1)
    )
    max_output = np.max(model.output[:, model.t > 1000])
    
    # check if stage 1 was successful
    # or np.max(model_pwrs) < 1 or np.median(model.output) < 1
    # info: filter of median<1 avoids down-to-up solutions
    # filtering median > 15 avoids finding solutions that stay in the up-state all the time
    if max_output > 160 or max_amp_output < 10 or ((np.median(model.output) < 1 or np.median(model.output) > 15) and MEDIAN_KILLER):
        return invalid_result, {}
    
    # Stage 2: full and final simulation
    # ---------------------------------------
    model.params['duration'] = defaultDuration
    model.run(chunkwise=True, bold = True, chunksize = int(1 * 60 * 1000 / model.params['dt'])) # 1 minute chunks
    
    # -------- fitness evaluation here --------
    fits = du.model_fit(model, ds, fcd=True if model.params.duration >= 5 * 60 * 1000 else False)
    if "fcd" not in fits:
        fits["fcd"] = 1
    
    scores = []
    scores.append(np.mean(fits['fc_scores']))
    scores.append(np.mean(fits['fcd']))
    
    # --- EEG
    
    model_frs, model_pwrs = func.getMeanPowerSpectrum(model.output, dt=model.params.dt, maxfr=40, spectrum_windowsize=10)
    
    # 1/f weighted correlation would be:
    # weights = [0.0] + list(1/model_frs[1:])
    # eeg_correlation = func.weighted_correlation(mean_eeg_power, model_pwrs, weights)
    # unweighted correlation:
    eeg_correlation = np.corrcoef(mean_eeg_power, model_pwrs)[0,1]

    domfr = model_frs[np.argmax(model_pwrs)]    
    scores.append(eeg_correlation)
    
    fitness_tuple = tuple(scores)
    return fitness_tuple, {
        "median_rate" : np.median(model.output),
        "output": model.output[:, ::int(model.params['save_dt']/model.params['dt'])],
        "model_pwrs" : model_pwrs
    }

# Evolution: No filtering (down-to-up)

In [138]:
MEDIAN_KILLER = False

In [32]:
pars = ParameterSpace(['mue_ext_mean', 'mui_ext_mean', 'Ke_gl', 'b', 'tauA', 'sigma_ou'], 
                      [[0.0, 4.0], [0.0, 4.0], [100.0, 400.0], [0.0, 20.0], [5.0, 5000.0], [0.0, 0.5]])

In [139]:
weightList = [1.0, -1.0, 1.0]
evolution = Evolution(evaluateSimulation, 
                      pars, 
                      weightList = weightList,                   
                      model = model, 
                      ncores=80,
                      POP_INIT_SIZE=640, 
                      POP_SIZE = 160, 
                      NGEN=50, 
                      algorithm='nsga2',
                      filename="evolution-9.0-EEG-no-median-killer.hdf")

MainProcess root INFO     Trajectory Name: results-2020-09-25-18H-45M-00S
MainProcess root INFO     Storing data to: /mnt/raid/data/cakan/hdf/evolution-9.0-EEG-no-median-killer.hdf
MainProcess root INFO     Trajectory Name: results-2020-09-25-18H-45M-00S
MainProcess root INFO     Number of cores: 80
MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `/mnt/raid/data/cakan/hdf/evolution-9.0-EEG-no-median-killer.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.
MainProcess root INFO     Evolution: Using algorithm: nsga2
/home/cakan/anaconda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/cakan/anaconda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwri

In [140]:
evolution.run(verbose = False)

MainProcess root INFO     Evaluating initial population of size 640 ...
MainProcess pypet.trajectory.Trajectory INFO     Your trajectory has not been explored, yet. I will call `f_explore` instead.
MainProcess pypet.environment.Environment INFO     I am preparing the Trajectory for the experiment and initialise the store.
MainProcess pypet.environment.Environment INFO     Initialising the storage for the trajectory.
MainProcess pypet.storageservice.HDF5StorageService INFO     Initialising storage or updating meta data of Trajectory `results-2020-09-25-18H-45M-00S`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Finished init or meta data update for `results-2020-09-25-18H-45M-00S`.
MainProcess pypet.environment.Environment INFO     
************************************************************
STARTING runs of trajectory
`results-2020-09-25-18H-45M-00S`.
************************************************************

MainProcess pypet.storageservice.HDF5StorageService INFO  

In [141]:
import dill
fname = os.path.join("/mnt/raid/data/cakan/dill/", "evolution-" + evolution.trajectoryName + ".dill")
print(f"Saving evolution to {fname}")
dill.dump(evolution, open(fname, "wb"))

Saving evolution to /mnt/raid/data/cakan/dill/evolution-results-2020-09-25-18H-45M-00S.dill


# With filtering (up-to-down)

In [149]:
# this turns on the filter
MEDIAN_KILLER = True

In [150]:
weightList = [1.0, -1.0, 1.0]
evolution = Evolution(evaluateSimulation, 
                      pars, 
                      weightList = weightList,                   
                      model = model, 
                      ncores=80,
                      POP_INIT_SIZE=640, 
                      POP_SIZE = 160, 
                      NGEN=50, 
                      algorithm='nsga2',
                      filename="evolution-9.0-EEG.hdf")

MainProcess root INFO     Trajectory Name: results-2020-09-27-12H-23M-15S
MainProcess root INFO     Storing data to: /mnt/raid/data/cakan/hdf/evolution-9.0-EEG.hdf
MainProcess root INFO     Trajectory Name: results-2020-09-27-12H-23M-15S
MainProcess root INFO     Number of cores: 80
MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `/mnt/raid/data/cakan/hdf/evolution-9.0-EEG.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.
MainProcess root INFO     Evolution: Using algorithm: nsga2
/home/cakan/anaconda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/cakan/anaconda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous c

In [151]:
evolution.run(verbose = False)

MainProcess root INFO     Evaluating initial population of size 640 ...
MainProcess pypet.trajectory.Trajectory INFO     Your trajectory has not been explored, yet. I will call `f_explore` instead.
MainProcess pypet.environment.Environment INFO     I am preparing the Trajectory for the experiment and initialise the store.
MainProcess pypet.environment.Environment INFO     Initialising the storage for the trajectory.
MainProcess pypet.storageservice.HDF5StorageService INFO     Initialising storage or updating meta data of Trajectory `results-2020-09-27-12H-23M-15S`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Finished init or meta data update for `results-2020-09-27-12H-23M-15S`.
MainProcess pypet.environment.Environment INFO     
************************************************************
STARTING runs of trajectory
`results-2020-09-27-12H-23M-15S`.
************************************************************

MainProcess pypet.storageservice.HDF5StorageService INFO  

In [152]:
import dill
fname = os.path.join("/mnt/raid/data/cakan/dill/", "evolution-" + evolution.trajectoryName + ".dill")
print(f"Saving evolution to {fname}")
dill.dump(evolution, open(fname, "wb"))

Saving evolution to /mnt/raid/data/cakan/dill/evolution-results-2020-09-27-12H-23M-15S.dill


# fMRI-only without adaptation

In [ ]:
MEDIAN_KILLER = False

In [ ]:
model.params['b'] = 0.0

In [ ]:
pars = ParameterSpace(['mue_ext_mean', 'mui_ext_mean', 'Ke_gl', 'sigma_ou'], 
                      [[0.0, 4.0], [0.0, 4.0], [100.0, 400.0], [0.0, 0.5]])

In [ ]:
weightList = [1.0, -1.0]
evolution = Evolution(evaluateSimulation, 
                      pars, 
                      weightList = weightList,                   
                      model = model, 
                      ncores=80,
                      POP_INIT_SIZE=320, 
                      POP_SIZE = 80, 
                      NGEN=20, 
                      algorithm='nsga2',
                      filename="evolution-test-9.0-b-0-fMRI-only.hdf")

In [ ]:
evolution.run(verbose = False)

In [ ]:
import dill
fname = os.path.join("/mnt/raid/data/cakan/dill/", "evolution-" + evolution.trajectoryName + ".dill")
print(f"Saving evolution to {fname}")
dill.dump(evolution, open(fname, "wb"))

# fMRI-only with adaptation

In [58]:
MEDIAN_KILLER = False

In [60]:
pars = ParameterSpace(['mue_ext_mean', 'mui_ext_mean', 'Ke_gl', 'b', 'tauA', 'sigma_ou'], 
                      [[0.0, 4.0], [0.0, 4.0], [100.0, 400.0], [0.0, 20.0], [5.0, 5000.0], [0.0, 0.5]])

In [61]:
weightList = [1.0, -1.0]
evolution = Evolution(evaluateSimulation, 
                      pars, 
                      weightList = weightList,                   
                      model = model, 
                      ncores=80,
                      POP_INIT_SIZE=320, 
                      POP_SIZE = 80, 
                      NGEN=20, 
                      algorithm='nsga2',
                      filename="evolution-test-9.0-fMRI-only.hdf")

MainProcess root INFO     Trajectory Name: results-2020-07-27-11H-49M-36S
MainProcess root INFO     Storing data to: /mnt/raid/data/cakan/hdf/evolution-test-9.0-fMRI-only.hdf
MainProcess root INFO     Trajectory Name: results-2020-07-27-11H-49M-36S
MainProcess root INFO     Number of cores: 80
MainProcess pypet.storageservice.HDF5StorageService INFO     I will use the hdf5 file `/mnt/raid/data/cakan/hdf/evolution-test-9.0-fMRI-only.hdf`.
MainProcess pypet.environment.Environment INFO     Environment initialized.
MainProcess root INFO     Evolution: Using algorithm: nsga2
/home/cakan/anaconda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/cakan/anaconda/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consid

In [62]:
evolution.run(verbose = False)

MainProcess root INFO     Evaluating initial population of size 320 ...
MainProcess pypet.trajectory.Trajectory INFO     Your trajectory has not been explored, yet. I will call `f_explore` instead.
MainProcess pypet.environment.Environment INFO     I am preparing the Trajectory for the experiment and initialise the store.
MainProcess pypet.environment.Environment INFO     Initialising the storage for the trajectory.
MainProcess pypet.storageservice.HDF5StorageService INFO     Initialising storage or updating meta data of Trajectory `results-2020-07-27-11H-49M-36S`.
MainProcess pypet.storageservice.HDF5StorageService INFO     Finished init or meta data update for `results-2020-07-27-11H-49M-36S`.
MainProcess pypet.environment.Environment INFO     
************************************************************
STARTING runs of trajectory
`results-2020-07-27-11H-49M-36S`.
************************************************************

MainProcess pypet.storageservice.HDF5StorageService INFO  

In [63]:
import dill
fname = os.path.join("/mnt/raid/data/cakan/dill/", "evolution-" + evolution.trajectoryName + ".dill")
print(f"Saving evolution to {fname}")
dill.dump(evolution, open(fname, "wb"))

Saving evolution to /mnt/raid/data/cakan/dill/evolution-results-2020-07-27-11H-49M-36S.dill
